In [7]:
import stanza, re, benepar, psutil, gc, json, csv, time, string

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [ ]:
# These only need to run once
# stanza.download('en')
spacy.cli.download("en_core_web_sm")
nltk.download('punkt_tab')
benepar.download('benepar_en3')

In [8]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go.txt")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


Select choice of text by specifying index (starting with 0):

In [9]:
text_choice = 1

In [10]:
file_path = directory_path+source_texts[text_choice]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day.txt


Try reading the file with different encodings:

In [11]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")

Failed to read with encoding: utf-8
Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
    Prologue: July 1956    Darlington Hall    IT seems increasingly likely that I really will undertake the expedition that has been preoccupying my imagination now for some days. An expedition, I should sa

Successfully read the file with encoding: windows-1252 (encoding index 3).  This gives the beginning as:
    Prologue: July 1956    Darlington Hall    IT seems increasingly likely that I really will undertake the expedition that has been preoccupying my imagination now for some days. An expedition, I should sa

Successfully read the file with encoding: iso-8859-1 (encoding index 4).  This gives the beginning as:
    Prologue: July 1956    Darlington Hall    IT seems increasingly likely that I really will undertake the expedition that has been preoccupying my imagination now for some days. An expedition, I should 

Are they the same?

In [83]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 423697:-
0 True
1 False
2 True
3 False
text 1 length 423697:-
0 False
1 True
2 False
3 False
text 2 length 423697:-
0 True
1 False
2 True
3 False
text 3 length 423697:-
0 False
1 False
2 False
3 True


Now load text file with selected encoding

In [82]:
print(len(texts[0]))

423697


In [9]:
encoding = encodings[2]
with open(file_path, 'r', encoding=encoding) as file:
    text = file.read()
print(f"Total characters in book:  {len(text)}")

Total characters in book:  423697


In [62]:
save_path = file_path[:-4]+" - edited.txt"
print(save_path)


C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited.txt


In [69]:
for t in texts:
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(save_path, 'r', encoding='utf-8') as file:
        text_edited = file.read()
    print(text_edited == t)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited.txt
True
True
True
True


In [34]:
text == text_edited

True

Watch out for this character as it is very awkward;

In [13]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

first print
142
second print



Identify invalid characters using string:

In [52]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Invalid characters found: {'É', 'Ó', 'Ò', '¤', 'Õ', '\t', 'á', 'Ð', '\n', '\x8e'}
Replacement dictionary: {'É': '?', 'Ó': '?', 'Ò': '?', '¤': '?', 'Õ': '?', '\t': '?', 'á': '?', 'Ð': '?', '\n': '?', '\x8e': '?'}
Replacement list of tuples: [('É', '?'), ('Ó', '?'), ('Ò', '?'), ('¤', '?'), ('Õ', '?'), ('\t', '?'), ('á', '?'), ('Ð', '?'), ('\n', '?'), ('\x8e', '?')]


Identify invalid characters using regular expression (re):

In [14]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-&]')

valid_encodings = [2, 3, 4, 5]
re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for latin-1: {'Ð', 'á', '~', '\x8e', 'Ò', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters using re for windows-1252: {'Ð', 'á', '~', 'Ò', 'Ž', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters using re for iso-8859-1: {'Ð', 'á', '~', '\x8e', 'Ò', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters using re for iso-8859-15: {'Ð', 'á', '€', '~', '\x8e', 'Ò', 'Õ', 'Ó', '}', 'É'}


In [15]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "é \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")
# Example text
#test_text = "This is a tÈst string with invàlid chàracters."
valid_encodings = [2, 3, 4, 5]
string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for latin-1: {'Ð', 'á', '~', '\x8e', 'Ò', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters found using string for windows-1252: {'Ð', 'á', '~', 'Ò', 'Ž', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters found using string for iso-8859-1: {'Ð', 'á', '~', '\x8e', 'Ò', 'Õ', '¤', 'Ó', '}', 'É'}
invalid characters found using string for iso-8859-15: {'Ð', 'á', '€', '~', '\x8e', 'Ò', 'Õ', 'Ó', '}', 'É'}


In [16]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

'abefgh'

In [17]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#&'()*+,-./:;<=>?@[\]^_`|é 
	
41
63
39
34
96


In [18]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [19]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding latin-1
10 invalid chars found using re   :  Ð  á  ~    Ò  Õ  ¤  Ó  }  É + unprintable character
10 invalid chars found using string:  Ð  á  ~    Ò  Õ  ¤  Ó  }  É + unprintable character

0 chars in re but not in string:    
0 chars in string but not in re:    

with encoding windows-1252
10 invalid chars found using re   :  Ð  á  ~  Ò  Ž  Õ  ¤  Ó  }  É
10 invalid chars found using string:  Ð  á  ~  Ò  Ž  Õ  ¤  Ó  }  É

0 chars in re but not in string:    
0 chars in string but not in re:    

with encoding iso-8859-1
10 invalid chars found using re   :  Ð  á  ~    Ò  Õ  ¤  Ó  }  É + unprintable character
10 invalid chars found using string:  Ð  á  ~    Ò  Õ  ¤  Ó  }  É + unprintable character

0 chars in re but not in string:    
0 chars in string but not in re:    

with encoding iso-8859-15
10 invalid chars found using re   :  Ð  á  €  ~    Ò  Õ  Ó  }  É + unprintable character
10 invalid chars found using string:  Ð  á  €  ~    Ò  Õ  Ó  }  É + unprintable charac

In [20]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

233
3


In [21]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        raise ValueError("Strings must be of the same length!")

# Create a list of tuples with differing characters
    differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
    print(differences[:100])

In [22]:
compare_texts(texts[2], texts[3])

[('¤', '€')]


Now we are going to display all the invalid characters in their context to decide what to do about them. We will do this for each of the different encodings.

In [30]:
def display_invalid_chars(test_text, window_size = 200):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " ()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' at position {pos}:\n{highlighted_window}\n")




In [31]:
display_invalid_chars(texts[0])

Invalid character '}' at position 104524:
s; but as I recall, his hair had greyed entirely and his tall slender figure already bore signs of t[}]:le stoop that was to become so pronounced in his last years. He barely glanced up from his volume 

Invalid character '~' at position 10340:
- including the back corridor, the two still rooms and the old laundry - and the guest corridor up o[~] the second floor would be dust-sheeted, leaving all the main ground-floor rooms and a generous num

Invalid character '~' at position 13782:
that Miss Kenton, with her great affection for this house, with her exemplary professionalism - the [~]ort almost impossible to find nowadays - was Just the factor needed to enable me to complete a full

Invalid character '~' at position 125575:
ite. I decided to minimize my presence by standing in the shadows much further away from table than [~] might usually have done. Of course, this strategy had a distinct disadvantage in that each time I 

Invalid character 

In [32]:
display_invalid_chars(texts[1])

Invalid character '}' at position 104524:
s; but as I recall, his hair had greyed entirely and his tall slender figure already bore signs of t[}]:le stoop that was to become so pronounced in his last years. He barely glanced up from his volume 

Invalid character '~' at position 10340:
- including the back corridor, the two still rooms and the old laundry - and the guest corridor up o[~] the second floor would be dust-sheeted, leaving all the main ground-floor rooms and a generous num

Invalid character '~' at position 13782:
that Miss Kenton, with her great affection for this house, with her exemplary professionalism - the [~]ort almost impossible to find nowadays - was Just the factor needed to enable me to complete a full

Invalid character '~' at position 125575:
ite. I decided to minimize my presence by standing in the shadows much further away from table than [~] might usually have done. Of course, this strategy had a distinct disadvantage in that each time I 

Invalid character 

In [33]:
display_invalid_chars(texts[2])

Invalid character '}' at position 104524:
s; but as I recall, his hair had greyed entirely and his tall slender figure already bore signs of t[}]:le stoop that was to become so pronounced in his last years. He barely glanced up from his volume 

Invalid character '~' at position 10340:
- including the back corridor, the two still rooms and the old laundry - and the guest corridor up o[~] the second floor would be dust-sheeted, leaving all the main ground-floor rooms and a generous num

Invalid character '~' at position 13782:
that Miss Kenton, with her great affection for this house, with her exemplary professionalism - the [~]ort almost impossible to find nowadays - was Just the factor needed to enable me to complete a full

Invalid character '~' at position 125575:
ite. I decided to minimize my presence by standing in the shadows much further away from table than [~] might usually have done. Of course, this strategy had a distinct disadvantage in that each time I 

Invalid character 

In [34]:
display_invalid_chars(texts[3])

Invalid character '}' at position 104524:
s; but as I recall, his hair had greyed entirely and his tall slender figure already bore signs of t[}]:le stoop that was to become so pronounced in his last years. He barely glanced up from his volume 

Invalid character '~' at position 10340:
- including the back corridor, the two still rooms and the old laundry - and the guest corridor up o[~] the second floor would be dust-sheeted, leaving all the main ground-floor rooms and a generous num

Invalid character '~' at position 13782:
that Miss Kenton, with her great affection for this house, with her exemplary professionalism - the [~]ort almost impossible to find nowadays - was Just the factor needed to enable me to complete a full

Invalid character '~' at position 125575:
ite. I decided to minimize my presence by standing in the shadows much further away from table than [~] might usually have done. Of course, this strategy had a distinct disadvantage in that each time I 

Invalid character 

In [36]:
print("€" in texts[3])
print(texts[3][246245])

True
€


In [38]:
print(ord(texts[2][146834]))

142


In [40]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            print(f"Before: {before}[{old}]{after}")
            print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    return text




In [41]:
replacements = dict()
replacements["t}:le"] = "the"
replacements["up o~ the"] = "up on the"
replacements["the ~ort"] = "the sort"
replacements["than ~ might"] = "than might"
replacements["the house~"] = "the house."
replacements["while ~earlier"] = "while earlier"
replacements["meetings ~once"] = "meetings once"
replacements["Ž"] = "é"
replacements["mean- it"] = "mean it"
replacements["¤"] = "s"
replacements["Ð"] = "-"
replacements["ÉÓ"] = '-"'
replacements["Ó"] = '"'
replacements["Ò"] = '"'
replacements["Rover.Ó I thanked"] = "Rover  I thanked"
replacements['Rover." I thanked'] = "Rover  I thanked"
replacements["Õ"] = "'"
replacements["á"] = ""


for key, value in replacements.items():
        print(f"{key}: {value}")

t}:le: the
up o~ the: up on the
the ~ort: the sort
than ~ might: than might
the house~: the house.
while ~earlier: while earlier
meetings ~once: meetings once
Ž: é
mean- it: mean it
¤: s
Ð: -
ÉÓ: -"
Ó: "
Ò: "
Rover.Ó I thanked: Rover  I thanked
Rover." I thanked: Rover  I thanked
Õ: '
á: 


In [42]:
text_edited = replace_strings_with_context(texts[1],replacements)

Before:  signs of [t}:le] stoop tha
After:   signs of [the] stoop tha

Before:  corridor [up o~ the] second fl
After:   corridor [up on the] second fl

Before: onalism - [the ~ort] almost im
After:  onalism - [the sort] almost im

Before: rom table [than ~ might] usually h
After:  rom table [than might] usually h

Before:  visit to [the house~] In fact, 
After:   visit to [the house.] In fact, 

Before:  a little [while ~earlier]. I can re
After:   a little [while earlier]. I can re

Before: e evening [meetings ~once] and for a
After:  e evening [meetings once] and for a

Before: the attach[Ž] case at h
After:  the attach[é] case at h

Before: his attach[Ž] case" - h
After:  his attach[é] case" - h

Before: his attach[Ž] case - wh
After:  his attach[é] case - wh

Before: his attach[Ž] case to h
After:  his attach[é] case to h

Before: his attach[Ž] case as u
After:  his attach[é] case as u

Before: his attach[Ž] case away
After:  his attach[é] case away

Before: aidÉÓ  "I [mean- it], M

Now see if there are any problems left:

In [45]:
display_invalid_chars(text_edited)


In [46]:
test_text.find("Prologue")

2

Next we get rid of the chapter headers

In [47]:
test_text=text_edited
chapters = list()
for title in ["Prologue", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Eleven"]:
    ind = test_text.find(title)
    while ind != -1:
        start = test_text[ind:ind + 100]
        start = start.replace("\n", " ")
        start = start.replace("\t", " ")
        print (start)
        chapters.append(title)
        ind = test_text.find(title, ind + 1)     
print(chapters)

Prologue: July 1956  Darlington Hall  IT seems increasingly likely that I really will undertake the 
One need hardly dwell on the catastrophic possibility of uttering a bantering remark only to discove
One - Evening  Salisbury  TONIGHT, I find myself here in a guest house in the city of Salisbury. The
One had to be guided by the judgement of 'the true ladies and gentlemen', argued the Society, or els
One afternoon, evidently, this butler had entered the dining room to make sure all was well for dinn
One such instance was related to me by Mr David Charles, of the Charles and Redding Company, who vis
One afternoon, Mr Charles to his shame and regret had allowed himself to become   inebriated in the 
One wishes for a good bath at the mere reminder."  "Yes, sir."   "Last time I was in Berlin, Stevens
One thing you can bet on for sure." He leaned forward and waved his cigar for emphasis. "Dupont hate
One might even have thought him to be angry. "We have heard in these two days several thoro

In [48]:
titles = list()
titles.append(("Prologue", "IT"))
titles.append(("Day One", "TONIGHT"))
titles.append(("Day Two", "STRANGE"))
titles.append(("Day Three - Morning", "LODGED"))
titles.append(("Day Three - Evening", "FEEL"))
titles.append(("Day Four", "HAVE"))
titles.append(("Day Six", "THIS"))

for edit_start, edit_end in titles:
    print(edit_start, edit_end)


Prologue IT
Day One TONIGHT
Day Two STRANGE
Day Three - Morning LODGED
Day Three - Evening FEEL
Day Four HAVE
Day Six THIS


In [49]:
for edit_start, edit_end in titles:
    
    ind1 = text_edited.find(edit_start)
    ind2 = text_edited.find(edit_end)
    if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
        print (f"removing {edit_start} to {edit_end}")
        text_edited = text_edited[:ind1] + text_edited[ind2:]
    else:
        print(f"{edit_start} to {edit_end} not removed")


removing Prologue to IT
removing Day One to TONIGHT
removing Day Two to STRANGE
removing Day Three - Morning to LODGED
removing Day Three - Evening to FEEL
removing Day Four to HAVE
removing Day Six to THIS


In [53]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words


In [52]:


X

capitalized_words = find_capitalized_words(texts[0])
for word in capitalized_words:
    if word != "OK":
        ind = text_edited.find(word)
        window = text_edited[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)



ving been to that part of the country, I would often glance through Volume III of Mrs Symons's work, the volume whi
 all, I can see no genuine reason 'Why I should not undertake this trip.   TONIGHT, I find myself here in a guest h
 each of us may better strive towards attaining 'dignity' for ourselves.   STRANGE beds have rarely agreed with me,

oday nothing but proud and grateful to have been given such a privilege.   LODGED last night in an inn named the Co
ounds from below of the landlord and his wife clearing up for the night.   FEEL I should perhaps return a moment to
uite illogical that I should feel any regret or shame on my own account.   HAVE finally arrived at Little Compton, 
lanations for that sense of triumph I came to be uplifted by that night.   THIS seaside town is a place I have thou


In [59]:
def display_text_in_context(source_text,target_text,window = 50):
    ind = source_text.find(target_text)
    if ind != -1:
        window = source_text[ind - window: ind + window + len(target_text)]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    else:
        print("not found")


In [60]:
display_text_in_context(text_edited, "been given such a privilege")

I am today nothing but proud and grateful to have been given such a privilege.   LODGED last night in an inn named the Coach an


In [378]:
print(text_edited[:20000])



IT seems increasingly likely that I really will undertake the expedition that has been preoccupying my imagination now for some days. An expedition, I should say, which I will undertake alone, in the comfort of Mr Farraday's Ford; an expedition which, as I foresee it, will take me through much of the finest countryside of England to the West Country, and may keep me away from Darlington Hall for as much as five or six days. The idea of such a journey came about, I should point out, from a most kind suggestion put to me by Mr Farraday himself one afternoon almost a fortnight ago, when I had been dusting the portraits in the library. In fact, as I recall, I was up on the step-ladder dusting the portrait of Viscount Wetherby when my employer had entered carrying a few volumes which he presumably wished returned to the shelves. On seeing my person, he took the opportunity to inform me that he had just that moment finalized plans to return to the United States for a period of five weeks b

In [63]:
save_path_test = save_path[:-4]+"1 test.txt"
print(save_path_test)




C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited1 test.txt


In [64]:
with open(save_path_test, 'w', encoding='utf-8') as f:
    f.write(text_edited)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [85]:
with open(save_path_test) as f:
    text_test = f.read()
with open(save_path) as f:
    text = f.read()

In [84]:
print(save_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited 1.txt


In [86]:
text == text_test

True

In [70]:
save_path = save_path[:-5]
save_path = save_path + " 1.txt"
print(save_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited 1.txt


In [74]:
text_edited.find("é")

146729

In [75]:
print(text[:146730] == text_edited[:146730])
print(len(text))
print(len(text_edited))

False
423412
423406


In [237]:

# Sample text with hyphenated words across lines
text = texts[3]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(text))


0
423697
